In [1]:
import os
import datetime
import time
from huggingface_hub import hf_hub_download, snapshot_download
from safetensors.torch import load_file
import torch

/home/dingfangyu/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = 'gpt2'
sf_filename = hf_hub_download(model_id, filename="model.safetensors")
pt_filename = hf_hub_download(model_id, filename="pytorch_model.bin")

In [3]:
from transformers import AutoModelForCausalLM

start_st = time.time()
weights = load_file(sf_filename, device="cpu")
dm = sum([v.numel() * v.dtype.itemsize for k, v in weights.items()])
s = sum([v.sum() for k, v in weights.items()]) 
# print(s)
# model = AutoModelForCausalLM.from_pretrained('gpt2', use_safetensors=True)
load_time_st = time.time() - start_st
print(f"Loaded safetensors {load_time_st:.3f} s, bdw {dm/load_time_st/2**30:.3f} GB/s")

start_pt = time.time()
weights = torch.load(pt_filename, map_location="cpu")
dm = sum([v.numel() * v.dtype.itemsize for k, v in weights.items()])
s = sum([v.sum() for k, v in weights.items()]) 
# print(s)
# model = AutoModelForCausalLM.from_pretrained('gpt2', use_safetensors=False)
load_time_pt = time.time() - start_pt
print(f"Loaded pytorch {load_time_pt:.3f} s, bdw {dm/load_time_pt/2**30:.3f} GB/s")

print(f"on CPU, safetensors is faster than pytorch by: {load_time_pt/load_time_st:.1f} X")

Loaded safetensors 0.070 s, bdw 7.322 GB/s
Loaded pytorch 0.258 s, bdw 1.981 GB/s
on CPU, safetensors is faster than pytorch by: 3.7 X


In [4]:
model = AutoModelForCausalLM.from_pretrained('gpt2', use_safetensors=True)
model = AutoModelForCausalLM.from_pretrained('gpt2', use_safetensors=False)